# 2 Data wrangling<a id='2_Data_wrangling'></a>

### 2.1 Importing Necessary Modules and Data

In [2]:
import pandas as pd
import os.path
from os import path

import warnings
warnings.filterwarnings("ignore")

### 2.2 Creating Features

Our data consists of 10 sensor readings place around volcanos. These 10 sensors have been normalized and separated into their own CSV file based on the volcano they were reading. The target data has been provided in a separate CSV file that links the segment ID(the name of the file our features are in) and the time to next eruption. Due to the size of our data(14GB) and the way the data is organized, we will create new features to consolidate as much data as we can into a smaller space and something that is better able to be used as an input for our models.

The raw data cannot be attached to this project but can be found at https://www.kaggle.com/c/predict-volcanic-eruptions-ingv-oe/data?select=train.csv
Place the train.csv and train folder in the predict-volcanic-eruptions folder but most of the project will focus on the Data.csv attached

In [3]:
targetData = pd.read_csv('predict-volcanic-eruptions/train.csv')

In [5]:
if not path.exists('predict-volcanic-eruptions/Data.csv'):
    columns = ['segment_id']
    featureFunctionSuffix = ['_mean','_01percentile','_05percentile','_10percentile','_20percentile','_25percentile','_35percentile','_45percentile','_65percentile','_75percentile','_80percentile','_90percentile','_99percentile','_95percentile','_std','_var','_max','_min','_range','_kurtosis','_skew','_sum','_meanAbsDev','_localMinCount','_localMaxCount','_isNullCount']
   
    for i in os.listdir('predict-volcanic-eruptions/train/') :
        rows = []
        rowsMissingData = []
        fileNameSplit = i.split('.')
        
        #Checks to make sure it only reads csv files in the folder since jupyter creates a checkpoint folder
        if fileNameSplit[1] != 'csv' :
            continue
        
        #Separates out the segmentID out of the filename to use with train.csv
        segmentID = fileNameSplit[0]
        
        #Appends the segmentID at the start of each row to use to merge
        rows.append(segmentID)
        rowsMissingData.append(segmentID)
        
        #load in the raw sensor data
        rawData = pd.read_csv('predict-volcanic-eruptions/train/' + i)
        rawData = rawData.astype('float32')
    
        #Calculate the column names with the suffic sensor_#_suffix if the columns list is empty
        if columns == ['segment_id']:
            for j in rawData.columns:
                for k in featureFunctionSuffix:
                    columns.append(j + k)
                    
            #Creates a temporary dataframe with the column names just to store all the rows o
            featureData = pd.DataFrame(columns = columns)
            missingData = pd.DataFrame(columns = ['Segment_ID','Sensor1','Sensor2','Sensor3','Sensor4','Sensor5','Sensor6','Sensor7','Sensor8','Sensor9','Sensor10'])
                    
    
        #Cleaning and creating features of the data
        for j in rawData.columns:
            #Creating a temporary dataframe of just a single sensor
            columnData = rawData[[j]]
            
            #Fill in missing values
            columnData = ((columnData.bfill() + columnData.ffill()) / 2)
            columnData = columnData.bfill().ffill()
            
            #Count null values in each sensor for each file
            rowsMissingData.append(columnData.isnull().sum()[0])
            
            #Creating features for a single sensor
            meanData = columnData.mean()
            percentile01Data = columnData.quantile(0.01)
            percentile05Data = columnData.quantile(0.05)
            percentile25Data = columnData.quantile(0.25)
            percentile75Data = columnData.quantile(0.75)
            percentile10Data = columnData.quantile(0.10)
            percentile20Data = columnData.quantile(0.20)
            percentile35Data = columnData.quantile(0.35)
            percentile45Data = columnData.quantile(0.45)
            percentile65Data = columnData.quantile(0.65)
            percentile80Data = columnData.quantile(0.80)
            percentile90Data = columnData.quantile(0.90)
            percentile99Data = columnData.quantile(0.99)
            percentile95Data = columnData.quantile(0.95)
            kurtosisData = columnData.kurtosis()
            stdData = columnData.std()
            maxData = columnData.max()
            minData = columnData.min()
            rangeData = maxData - minData
            skewData = columnData.skew()
            sumData = columnData.sum()
            varData = columnData.var()
            madData = columnData.mad()
            localMinData = columnData[(columnData.shift(1) > columnData) & (columnData.shift(-1) > columnData)].count()
            localMaxData = columnData[(columnData.shift(1) < columnData) & (columnData.shift(-1) < columnData)].count()
            isNullData = columnData.isna().sum()
            
            rows.append(meanData[0])
            rows.append(percentile01Data[0])
            rows.append(percentile05Data[0])
            rows.append(percentile10Data[0])
            rows.append(percentile20Data[0])
            rows.append(percentile25Data[0])
            rows.append(percentile35Data[0])
            rows.append(percentile45Data[0])
            rows.append(percentile65Data[0])
            rows.append(percentile75Data[0])
            rows.append(percentile80Data[0])
            rows.append(percentile90Data[0])
            rows.append(percentile95Data[0])
            rows.append(percentile99Data[0])
            rows.append(stdData[0])
            rows.append(varData[0])
            rows.append(maxData[0])
            rows.append(minData[0])
            rows.append(rangeData[0])
            rows.append(kurtosisData[0])
            rows.append(skewData[0])
            rows.append(sumData[0])
            rows.append(madData[0])
            rows.append(localMinData[0])
            rows.append(localMaxData[0])
            rows.append(isNullData[0])
        
        featureData.loc[len(featureData)] = rows
        missingData.loc[len(missingData)] = rowsMissingData
    
    #Merging the feature and target data into a full single dataframe
    featureData['segment_id'] = featureData['segment_id'].astype('int64')
    fullData = featureData.merge(targetData,on='segment_id')
    
    #Exporting to csv for future use
    fullData.to_csv('predict-volcanic-eruptions/Data.csv',index = False)
    missingData.to_csv('predict-volcanic-eruptions/MissingData.csv',index = False)

We have create our own features consisting of some basic aggregate functions, such as mean, median and range. As well as others such as percentiles at many different locations, the number of peaks in each file and the count of null data that sensors may have. After turning all our data into features, we have 272 columns. 271 columns are features and 1 column target, with 4431 rows of data. This is good because that's how many files were provided.

In [6]:
missingDataSum = pd.read_csv('predict-volcanic-eruptions/MissingData.csv')
data = pd.read_csv('predict-volcanic-eruptions/Data.csv')
data.head()

,segment_id,sensor_1_mean,sensor_1_01percentile,sensor_1_05percentile,sensor_1_10percentile,sensor_1_20percentile,sensor_1_25percentile,sensor_1_35percentile,sensor_1_45percentile,sensor_1_65percentile,...,sensor_10_min,sensor_10_range,sensor_10_kurtosis,sensor_10_skew,sensor_10_sum,sensor_10_meanAbsDev,sensor_10_localMinCount,sensor_10_localMaxCount,sensor_10_isNullCount,time_to_eruption
0,1000015382,0.382244,-277.0,-174.0,-130.0,-83.0,-66.0,-38.0,-12.0,37.0,...,-2961.0,6140.0,14.978788,0.058227,53806.0,163.679382,8368,8380,0,16258654
1,1000554676,-3.828120,-1252.0,-878.0,-686.0,-446.0,-356.0,-206.0,-67.0,199.0,...,-4329.0,8771.0,0.160791,0.004739,-445008.0,835.125977,2727,2725,0,6347792
2,1000745424,8.291928,-1392.0,-989.0,-765.0,-497.0,-400.0,-230.0,-74.0,234.0,...,-5040.0,10270.0,0.193508,-0.025480,-89519.0,972.496460,2347,2358,0,5120693
3,1001461087,2.071582,-1017.0,-645.0,-485.0,-310.0,-246.0,-141.0,-46.0,140.0,...,-4634.0,10422.0,2.732420,-0.051502,-82408.0,548.506348,4988,4995,0,10393161
4,1001732002,0.904102,-702.0,-465.0,-358.0,-233.0,-187.0,-104.0,-33.0,105.0,...,-3909.0,8483.0,1.135692,0.375558,1922895.0,691.087891,2808,2825,0,20549733


In [7]:
data.describe()

,segment_id,sensor_1_mean,sensor_1_01percentile,sensor_1_05percentile,sensor_1_10percentile,sensor_1_20percentile,sensor_1_25percentile,sensor_1_35percentile,sensor_1_45percentile,sensor_1_65percentile,...,sensor_10_min,sensor_10_range,sensor_10_kurtosis,sensor_10_skew,sensor_10_sum,sensor_10_meanAbsDev,sensor_10_localMinCount,sensor_10_localMaxCount,sensor_10_isNullCount,time_to_eruption
count,4.431000e+03,4407.000000,4407.000000,4407.000000,4407.000000,4407.000000,4407.000000,4407.000000,4407.000000,4407.000000,...,4407.000000,4407.000000,4407.000000,4407.000000,4.431000e+03,4407.000000,4431.000000,4431.000000,4431.00000,4.431000e+03
mean,1.074694e+09,-0.496326,-1587.182891,-1055.044929,-806.991831,-520.655321,-415.220331,-235.737690,-75.912866,235.188564,...,-5276.923304,10533.836397,2.615635,-0.001230,-4.537815e+04,821.753609,4713.672986,4713.413676,324.98849,2.284891e+07
std,6.161966e+08,18.048715,3494.870497,2529.440739,2015.300603,1301.859271,1035.721435,588.790749,189.820954,586.357329,...,5208.049716,10341.172094,8.776035,0.120686,3.242788e+06,1471.698499,3045.610966,3045.715084,4404.36088,1.348439e+07
min,5.131810e+05,-595.469238,-32767.000000,-32767.000000,-32767.000000,-21859.000000,-17308.000000,-9715.000000,-3040.000000,32.000000,...,-32767.000000,2050.000000,-1.402664,-1.425080,-1.812419e+08,137.147858,0.000000,0.000000,0.00000,6.250000e+03
25%,5.527934e+08,-1.795862,-1307.500000,-885.000000,-676.000000,-437.000000,-347.500000,-197.000000,-64.000000,97.000000,...,-5169.500000,6296.500000,0.305904,-0.036110,-2.007085e+05,441.065979,2656.000000,2652.000000,0.00000,1.127016e+07
50%,1.066153e+09,0.026116,-882.000000,-580.000000,-443.000000,-287.000000,-229.000000,-130.000000,-42.000000,130.000000,...,-4126.000000,8271.000000,0.799168,-0.001341,5.633000e+03,556.892517,3349.000000,3344.000000,0.00000,2.246559e+07
75%,1.606350e+09,1.842786,-675.000000,-441.000000,-335.000000,-217.000000,-173.000000,-98.000000,-31.000000,198.000000,...,-3117.000000,10267.000000,2.360290,0.033720,2.291995e+05,756.923187,5911.500000,5910.500000,0.00000,3.434356e+07
max,2.146939e+09,341.219940,-232.000000,-152.000000,0.000000,127.000000,127.000000,127.000000,127.000000,9772.000000,...,-1004.000000,65534.000000,234.337540,1.775161,6.134024e+07,20721.322266,17408.000000,17407.000000,60001.00000,4.904609e+07


In [8]:
filterDF = ((missingDataSum['Sensor1'] < 60001) & (missingDataSum['Sensor1'] > 0)) | ((missingDataSum['Sensor2'] < 60001) & (missingDataSum['Sensor2'] > 0)) | ((missingDataSum['Sensor3'] < 60001) & (missingDataSum['Sensor3'] > 0)) | ((missingDataSum['Sensor4'] < 60001) & (missingDataSum['Sensor4'] > 0)) | ((missingDataSum['Sensor5'] < 60001) & (missingDataSum['Sensor5'] > 0)) | ((missingDataSum['Sensor6'] < 60001) & (missingDataSum['Sensor6'] > 0)) | ((missingDataSum['Sensor7'] < 60001) & (missingDataSum['Sensor7'] > 0)) | ((missingDataSum['Sensor8'] < 60001) & (missingDataSum['Sensor8'] > 0)) | ((missingDataSum['Sensor9'] < 60001) & (missingDataSum['Sensor9'] > 0)) | ((missingDataSum['Sensor10'] < 60001) & (missingDataSum['Sensor10'] > 0))
missingDataSum[filterDF].head()

,Segment_ID,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10


In [9]:
droppedData = data.dropna(how = 'any', axis = 0)
droppedData.describe()

,segment_id,sensor_1_mean,sensor_1_01percentile,sensor_1_05percentile,sensor_1_10percentile,sensor_1_20percentile,sensor_1_25percentile,sensor_1_35percentile,sensor_1_45percentile,sensor_1_65percentile,...,sensor_10_min,sensor_10_range,sensor_10_kurtosis,sensor_10_skew,sensor_10_sum,sensor_10_meanAbsDev,sensor_10_localMinCount,sensor_10_localMaxCount,sensor_10_isNullCount,time_to_eruption
count,2.705000e+03,2705.000000,2705.000000,2705.000000,2705.000000,2705.000000,2705.000000,2705.000000,2705.000000,2705.000000,...,2705.000000,2705.000000,2705.000000,2705.000000,2.705000e+03,2705.000000,2705.000000,2705.00000,2705.0,2.705000e+03
mean,1.074980e+09,-0.809179,-1915.634381,-1290.573752,-990.202588,-639.246580,-509.884288,-289.601479,-93.458410,288.689464,...,-5695.756377,11393.314972,2.419269,0.002877,-5.392732e+04,969.722144,4452.006285,4451.92939,0.0,2.067818e+07
std,6.166767e+08,22.448064,4332.532069,3149.901649,2514.072768,1624.912842,1292.700352,734.858705,236.947275,732.008965,...,6142.149438,12254.290189,9.586602,0.130629,4.095028e+06,1826.020800,2928.586522,2928.40006,0.0,1.376247e+07
min,5.131810e+05,-595.469238,-32767.000000,-32767.000000,-32767.000000,-21859.000000,-17308.000000,-9715.000000,-3040.000000,32.000000,...,-32767.000000,2050.000000,-1.402664,-1.425080,-1.812419e+08,137.147858,1062.000000,1079.00000,0.0,2.573000e+04
25%,5.498327e+08,-1.870569,-1383.000000,-943.000000,-723.000000,-467.000000,-372.000000,-212.000000,-68.000000,98.000000,...,-5242.000000,6251.000000,0.313780,-0.034150,-1.891330e+05,459.236420,2591.000000,2592.00000,0.0,8.213206e+06
50%,1.061557e+09,-0.045199,-895.000000,-598.000000,-457.000000,-296.000000,-237.000000,-134.000000,-43.000000,134.000000,...,-4170.000000,8362.000000,0.781562,-0.000070,1.873200e+04,599.190857,3011.000000,3010.00000,0.0,1.961127e+07
75%,1.606406e+09,1.798603,-662.000000,-442.000000,-338.000000,-220.000000,-175.000000,-99.000000,-32.000000,212.000000,...,-3080.000000,10501.000000,1.952938,0.035355,2.668250e+05,793.831787,5541.000000,5542.00000,0.0,3.238338e+07
max,2.146939e+09,341.219940,-232.000000,-152.000000,-114.000000,-71.000000,-57.000000,-32.000000,-9.000000,9772.000000,...,-1004.000000,65534.000000,234.337540,1.775161,6.134024e+07,20721.322266,16886.000000,16886.00000,0.0,4.795107e+07
